# Activity: Debug our Fibonacci Calculation
In this activity, we'll continue the development of our Fibonacci sequence calculations. Previously, we constructed an empty `MyFibonacciSequenceModel` struct instance, by calling the default `MyFibonacciSequenceModel()` constructor method. We then passed this model instance to the `fibonacci!(...)` method to compute the Fibonacci numbers for a given index. This worked fine, but we can do better!

We'll formulate a `build(...)` method which is responsible for properly constructing  `MyFibonacciSequenceModel` instances, and we'll make sure the `fibonacci!(...)` method is robust to bad inputs, to ensure it works correctly, and responds gracefully to errors.

### Review
 A [Fibonacci sequence](https://en.wikipedia.org/wiki/Fibonacci_sequence) is composed of the Fibonacci numbers $F_{n}$ where:
$$
\begin{align*}
F_{0} & = 0 \quad n = 0\\
F_{1} & = 1 \quad n = 1\\
F_{n} & = F_{n-2} + F_{n-1}\quad{n\geq{2}}
\end{align*}
$$

Let's implement a `build(...)` method to create a `MyFibonacciSequenceModel` instance with a specified size, and default values for the other arguments. We'll then pass this instance to the `fibonacci!(...)` method to compute the Fibonacci numbers for a given index.

Let's go!

___


## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.
* The [include command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 
* In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl), check out that documentation for more information on the functions and types used in this material.

In [1]:
include("Include.jl"); # load the Include.jl file

### Types and Functions
Let's define the `MyFibonacciSequenceModel` type, and some no data iteration tagging types (which tells us which iteration method to use). 

In [2]:
abstract type AbstractSequenceModel end # This is an abstract type for sequence models
abstract type AbstractIterationModel end # This is an abstract type for iteration models

"""
    MyFibonacciSequenceModel <: AbstractSequenceModel

A mutable struct to represent a Fibonacci sequence. 

### Fields
- `n::Int64`: The number of elements in the sequence.
- `sequence::Dict{Int64, Int64}`: The sequence itself, stored as a dictionary with indices as keys and Fibonacci numbers as values.

"""
mutable struct MyFibonacciSequenceModel <: AbstractSequenceModel

    # data -
    n::Int64 # number of elements in the sequence
    sequence::Dict{Int64, Int64} # the sequence itself

    # constructor -
    MyFibonacciSequenceModel() = new();
end

"""
    MyForLoopIterationModel <: AbstractIterationModel

An immutable struct to represent a for loop iteration model. This type has no fields and serves as a marker for a for loop iteration implementation.
"""
struct MyForLoopIterationModel <: AbstractIterationModel
    MyForLoopIterationModel() = new();
end

"""
    MyWhileLoopIterationModel <: AbstractIterationModel

An immutable struct to represent a while loop iteration model. This type has no fields and serves as a marker for a while loop iteration implementation.
"""
struct MyWhileLoopIterationModel <: AbstractIterationModel
    MyWhileLoopIterationModel() = new();
end

MyWhileLoopIterationModel

The `fibonacci!(...)` method will compute the Fibonacci numbers for a given index, given an instance of `MyFibonacciSequenceModel`. Remember, the `!` at the end of the method name indicates that this method will modify the input model instance in place.
* _Mutating methods_: In Julia, a `!` at the end of a function name indicates that the function _modifies its arguments_ in some way that will be visible after the method execution has ended. In this case, the `fibonacci!` method modifies the `my_sequence_model` by populating its `sequence` field with Fibonacci numbers.
* _Is there something magical about the `!` character_? No adding the `!` character to the end of function name is _not magic_. It's just a convention to help identify functions that may change state or data outide the local scope of the function. In this particular case, the `fibonacci!` method modifies the `my_sequence_model` by populating its `sequence` field with Fibonacci numbers.
* _Optional keyword arguments_: The `iterationmodel::T` optional argument defaults to an instance of `MyForLoopIterationModel`, thus, we'll use the for-loop iteration method by default. If we wanted to use our while loop implementation, should we pass in a `MyWhileLoopIterationModel` instance.

In [3]:
# -- PRIVATE METHODS BELOW HERE ------------------------------------------------------------------------------------------------------- #
function _fibonacci(sequencemodel::MyFibonacciSequenceModel, iterationmodel::MyForLoopIterationModel)

    @info "Debug message: We are using the for loop iteration model"

    # initialize -
    n = sequencemodel.n;
    sequence = Dict{Int64, Int64}();

    # we know the first two elements -
    sequence[0] = 0;
    sequence[1] = 1;

    # main loop, compute F₂, ....
    for i ∈ 2:n # what is this short-hand for?
        sequence[i] = sequence[i-1] + sequence[i-2]
    end

    # update the model -
    sequencemodel.sequence = sequence;
end

function _fibonacci(sequencemodel::MyFibonacciSequenceModel, iterationmodel::MyWhileLoopIterationModel)

    @info "Debug message: We are using the while loop iteration model"

    # check: is n legit?
    n = sequencemodel.n;
    sequence = Dict{Int64, Int64}();
    
    # main loop 
    should_loop_continue = true
    i = 0;
    while (should_loop_continue == true)
       
        # conditional logic: hardcode 0, 1 else gets all other cases
        if (i == 0)
            sequence[i] = 0; 
        elseif (i == 1)
            sequence[i] = 1;
        else
            sequence[i] = sequence[i - 1] + sequence[i - 2]
        end

        # update i -
        i += 1; # this is short-hand for i = i + 1

        # check: should we go around again?
        if (i>n)
            should_loop_continue = false;
        end
    end
    
    # update the model -
    sequencemodel.sequence = sequence;
end
# -- PRIVATE METHODS ABOVE HERE ------------------------------------------------------------------------------------------------------- #

# -- PUBLIC METHODS BELOW HERE -------------------------------------------------------------------------------------------------------- #
"""
    function fibonacci!(sequencemodel::MyFibonacciSequenceModel; 
        iterationmodel::T = MyForLoopIterationModel()) where T <: AbstractIterationModel

This function computes the Fibonacci sequence, given sequence and iteration models. 
The sequence model is updated in place (the sequence model is mutable). 
The iteration model is used to determine the type of loop to use.

# Arguments
- `sequencemodel::MyFibonacciSequenceModel`: The sequence model to update. The sequence model must have a field `n::Int64` that is the number of elements to compute.
- `iterationmodel::T`: The iteration model to use. It must be a subtype of `AbstractIterationModel`. The default is `MyForLoopIterationModel`.

There is no return value. The `sequencemodel` is updated in place.
"""
function fibonacci!(sequencemodel::MyFibonacciSequenceModel; 
    iterationmodel::T = MyForLoopIterationModel()) where T <: AbstractIterationModel
    
    # Status: If we get here, then we know n >= 0
    _fibonacci(sequencemodel, iterationmodel); # multiple dispatch to the appropriate implementation
end;

## Task 1: Create a build method for MyFibonacciSequenceModel
In this task, we will create a `build(...)` method that constructs a `MyFibonacciSequenceModel` instance with a specified size and default values for the other fields. This method will ensure that the model is properly initialized before being passed to the `fibonacci!(...)` method.

__Requirements__:
* _Arguments_: The `build(...)` method will takes the type of thing we want to build, i.e., `MyFibonacciSequenceModel` , the sequence size `n::Int` and the default value `defaultvalue::Int` parameters will be passed in a `data::NamedTuple` instance. The `build(...)` method returns a properly constructed `MyFibonacciSequenceModel` instance.
* _Error conditions_: Fill me in

Ready, set go!

In [44]:
"""
    build(modeltype::Type{MyFibonacciSequenceModel}, data::NamedTuple) -> MyFibonacciSequenceModel

This function builds a new `MyFibonacciSequenceModel` from a named tuple containing the parameters for the model.

### Arguments
- `modeltype::Type{MyFibonacciSequenceModel}`: The type of the model to build. This is used to create an instance of the model.
- `data::NamedTuple`: A named tuple containing the parameters for the model. The named tuple must contain the fields `n` and `defaultvalue`.

### Returns
- `MyFibonacciSequenceModel`: A new instance of `MyFibonacciSequenceModel` with the parameters set from the named tuple.
"""
function build(modeltype::Type{MyFibonacciSequenceModel}, data::NamedTuple)::Union{MyFibonacciSequenceModel, Nothing}
    
    # Initially the model is nothing -
    sequencemodel = nothing;
    
    # TODO: Uncomment the following code to give a warning if the named tuple is missing required fields
    required_fields = [:n, :defaultvalue] # we must have these fields to build the model
    for field ∈ required_fields
        if haskey(data, field) == false
            @error "Ooops! Missing required field: $field. Cannot build the model, returning nothing."
            return nothing; # Early retrn we cannot build the model, so return nothing
        end
    end

    # we have the required fields, so we can build the model - build an empty model
    sequencemodel = modeltype(); # create an instance of the model type

    # TODO: Check the size parameter n
    # TODO: If we get here, then we know that the named tuple has the required fields
    # TODO: However, the values could be invalid, so we should check them
    # TODO: Uncomment the following code to check the value for for the size parameter n
    default_size_parameter = 10; # this is the default value for n
    if data.n isa Int64 && data.n >= 0
        sequencemodel.n = data.n; # set the number of elements in the sequence
    else
        @warn "Ooops! Invalid value for n: $(data.n). Using default value: $default_size_parameter."
        sequencemodel.n = default_size_parameter; # set the default value
    end

    # TODO: Check the defaultvalue parameter
    # TODO: Uncomment the following code to check the value for the defaultvalue parameter
    my_default_value = 0; # this is the default value for the defaultvalue parameter
    if data.defaultvalue isa Int64
        my_default_value = data.defaultvalue;
    else
        @warn "Ooops! Invalid value for defaultvalue: $(data.defaultvalue). Using default value: $my_default_value."
    end

    # TODO: Populate a default sequence disctionary with the default value
    # TODO: Initialize the sequence dictionary with the default value
    initial_sequence_dictionary = Dict{Int64, Int64}();
    for i in 0:(sequencemodel.n - 1)
        initial_sequence_dictionary[i] = my_default_value; # set the default value for each element
    end
    sequencemodel.sequence = initial_sequence_dictionary; # set the initial sequence

    # return the model -
    return sequencemodel; 
end;

## Task 2: Let's test our build method
In this task, we will test the `build(...)` method to ensure it correctly constructs a `MyFibonacciSequenceModel` instance with the specified size and default values. 

We'll consider several deifferent test cases, wehre different values of `n` and `defaultvalue` are used to create the model instance. Sommetimes these values will be valid, and sometimes they will not. However, our `build(...)` method should handle these cases gracefully, returning a valid model instance or throwing an appropriate error.

### Happy Path
Let's start with the first case, the so called _happy path_, where we provide valid size and defaultvalue arguments to the `build(...)` method. This should return a properly constructed `MyFibonacciSequenceModel` instance:

In [13]:
my_sequence_model = build(MyFibonacciSequenceModel, (n=10, defaultvalue=0)); # build a new model

This seems to have run! But let's do a few check to make sure we are doing what we think we are doing. 

* _Is the `my_sequence_model` instance the correct type?_ Let's check this using [the `isa(...)` method](https://docs.julialang.org/en/v1/base/base/#Core.isa) in combination with [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to ensure that the `my_sequence_model` instance is of type `MyFibonacciSequenceModel`.
* _Is the sequence dictionary initialized correctly?_ The `sequence::Dict{Int64,Int64}` dictionary should have a length equal to `n`, and all elements should be equal to the `defaultvalue`. We can use the [length(...) method](https://docs.julialang.org/en/v1/base/collections/#Base.length) to check the length of the `sequence` field, and [the all(...) method](https://docs.julialang.org/en/v1/base/collections/#Base.all-Tuple%7BAny%7D) to check that all elements are equal to the `defaultvalue`.

If any of these checks fail, [an `AssertionError`](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is thrown with a descriptive message.

In [14]:
let

    # Check 1: Check the model type -
    @assert my_sequence_model isa MyFibonacciSequenceModel "Oopps! The model is NOT of type MyFibonacciSequenceModel";

    # Check 2: Check the sequence length and default values -
    @assert length(my_sequence_model.sequence) == 10 "Oopps! The sequence length is NOT equal to 10.";
    @assert all(k -> my_sequence_model.sequence[k] == 0, keys(my_sequence_model.sequence)) "Oopps! Not all sequence values are equal to 0.";
end

### Error Handling
If all our checks pass, then we know that when provided with the correct parameters, the model is built correctly. However, sometimes users make mistakes, they can't read our mind (or we did a bad job of documenting the code). So, let's also test some error cases where users enters invalid parameters.

Right now (with all the debugging checks disabled), the `build(...)` method will blow up if we did not provide a valid `n` or `defaultvalue` parameter. What happens of we do not provide a `defaultvalue` parameter in `data::NamedTuple`? 

In [15]:
my_sequence_model = build(MyFibonacciSequenceModel, (n=10,)); # build a new model|

ErrorException: type NamedTuple has no field defaultvalue

Without the `defaultvalue` parameter, the `build(...)` method will throw the error: __type NamedTuple has no field defaultvalue__ Let's handle this gracefully by that all parameters are provided. 
* `Uncomment` the first error handling block in the `build(...)` method, and reload the method. The error checking logic checks that the required fields are provided in the `data::NamedTuple` instance. If a field is missing, an `ArgumentError` with a descriptive message is thrown.

Rerun the updated `build(...)` method with a `data::NamedTuple` instance that does not contain the `defaultvalue` field. What happens now?

In [30]:
my_sequence_model = build(MyFibonacciSequenceModel, (n=10,)); # build a new model

┌ Error: Ooops! Missing required field: defultvalue. Cannot build the model, returning nothing.
└ @ Main /Users/jeffreyvarner/Desktop/julia_work/CHEME-140-eCornell-Repository/CHEME-140-eCornell-Repository/courses/CHEME-141/module-2/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X11sZmlsZQ==.jl:22


Confirm the model is not built, and the `my_sequence_model` instance is [`nothing`](https://docs.julialang.org/en/v1/base/constants/#Core.nothing) as expected. We can use the [`isnothing(...)`](https://docs.julialang.org/en/v1/base/base/#Base.isnothing) method in combination with [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check if `my_sequence_model` is `nothing`.

In [34]:
@assert isnothing(my_sequence_model) == true "Oopps! The sequence is nothing, it should be a dictionary.";

Next, let's check what happens if we provide an invalid `n` parameter. For example, if we provide a negative value for `n`, the `build(...)` method should throw an `ArgumentError` with a descriptive message. 
* Without the next block of error checking logic, the `build(...)` method returns an empty `MyFibonacciSequenceModel` instance when passed bad values for the `n` parameter (or the `defaultvalue` parameter), which is not what we want (this is a strange state for the model to be in).

`Uncomment` the second and third error handling blocks in the `build(...)` method, and reload the method. The error checking logic checks that the `n` parameter is a positive integer, and that the `defaultvalue` parameter is an integer. If either of these conditions is not met, an `warning` message is shown.

In [38]:
my_sequence_model = build(MyFibonacciSequenceModel, (n=-10, defaultvalue = -1)); # build a new model

┌ Warning: Ooops! Invalid value for n: -10. Using default value: 10.
└ @ Main /Users/jeffreyvarner/Desktop/julia_work/CHEME-140-eCornell-Repository/CHEME-140-eCornell-Repository/courses/CHEME-141/module-2/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X11sZmlsZQ==.jl:38


While this works a little better, we still have a problem. The `build(...)` method returns an empty `MyFibonacciSequenceModel` instance when passed bad values for the `n` parameter, a default value for `n` is set on the model, but the `sequence::Dict{Int64,Int64}` dictionary is empty, which is not what we want (this is a strange state for the model to be in).

In [42]:
my_sequence_model.sequence

Dict{Int64, Int64}()

`Uncomment` the last error handling block in the `build(...)` method, and reload the method. The final error checking block fills the empry `sequence::Dict{Int64,Int64}` dictionary with the `defaultvalue` for all keys from `0` to `n-1`. 

In [45]:
my_sequence_model = build(MyFibonacciSequenceModel, (n=-10, defaultvalue = -1)); # build a new model

┌ Warning: Ooops! Invalid value for n: -10. Using default value: 10.
└ @ Main /Users/jeffreyvarner/Desktop/julia_work/CHEME-140-eCornell-Repository/CHEME-140-eCornell-Repository/courses/CHEME-141/module-2/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X11sZmlsZQ==.jl:38


We recieved a `warning` message, but a warning is not an error, so the `my_sequence_model` instance is still created, and we can use it to compute the Fibonacci numbers! Let's check the instance that gets returned by the `build(...)` method.
* _If correct_: If the `my_sequence_model` instance is not `nothing`, the model will be type `MyFibonacciSequenceModel`, and the `sequence::Dict{Int64,Int64}` dictionary should be populated with the default value for all keys from `0` to `n-1`.

Let's check these four conditions using [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert):

In [48]:
let

    # initalize -
    correct_sequence_length = 10; # this is the default value for n
    correct_default_value = -1; # this is the default value for the defaultvalue parameter

    # Check 1: Check the model type -
    @assert isnothing(my_sequence_model) == false "Oopps! The model is nothing, it should be a MyFibonacciSequenceModel.";
    @assert my_sequence_model isa MyFibonacciSequenceModel "Oopps! The model is NOT of type MyFibonacciSequenceModel";

    # Check 2: Check the sequence length and default values -
    @assert length(my_sequence_model.sequence) == correct_sequence_length "Oopps! The sequence length is NOT equal to 10.";
    @assert all(k -> my_sequence_model.sequence[k] == correct_default_value, keys(my_sequence_model.sequence)) "Oopps! Not all sequence values are equal to 0.";
end